# Bootstrap Your Own Latent (BYOL)

In this session we are going to implement Bootstrap Your Own Latent paper (https://arxiv.org/abs/2006.07733).

It uses a MoCo-style training (with asymmetric SiameseNet) but with a L2 loss penalty (it is not a contrastive-base method).

In [46]:
import os
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision.io import read_image

import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

from PIL import Image
import copy

# target == teacher

In [47]:
class CustomImageDatasetBis(Dataset):
    def __init__(self, data, targets=None, transform=None, target_transform=None):
        self.imgs = data # Tensore di tutte le immagini
        self.targets = targets
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img = self.imgs[idx] # Sampling randomico di emlementi del dataset
        if isinstance(img, str): # Può capitare che il dataset sia salvato come stringhe/path (da usare quando non è possibile salvarsi tutto il tensore del dataset)
          img = read_image(img_path) # Fuzione di Torchvision, trova un'immaigne dal path fornito
        else:
          img = Image.fromarray(img.astype('uint8'), 'RGB')
        label = self.targets[idx] # Non utile nel caso di self-supervised ovviamente
        if self.transform:
            img1 = self.transform(img) # Utilizzo le trasformazioni
            img2 = self.transform(img)
            # img = self.transform(img)  Già così genero due immagini augmented diverse, siccome le funzioni che trasformano sono randomiche (TODO, rivedi le variabili)
        if self.target_transform:
            label1 = self.target_transform(label)
            label2 = self.target_transform(label)
        else:
            label1 = label
            label2 = label
        return img1, img2# , label1, label2 # Concateno immaigni e labels
    
# simclr DA pipeline
s=1
size=32
color_jitter = transforms.ColorJitter(0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s)
transform = transforms.Compose([transforms.RandomResizedCrop(size=size),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.RandomApply([color_jitter], p=0.8),
                                  transforms.RandomGrayscale(p=0.2),
                                  transforms.GaussianBlur(kernel_size=3),
                                  transforms.ToTensor()])
    

In [48]:
data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True)   
trainset = CustomImageDatasetBis(data.data, data.targets, transform=transform)
dataloader = DataLoader(trainset, batch_size=64, shuffle=True, pin_memory=True)# fast computation



Files already downloaded and verified


In [49]:
#! scrivi come funzioan training loop di byol

# loss fn
def loss_fn(x, y):# equivalente alla distanza suller slide per byol, equivalente a cosine similarity...
    x = F.normalize(x, dim=-1, p=2)
    y = F.normalize(y, dim=-1, p=2)
    return 2 - 2 * (x * y).sum(dim=-1)# proporzionale to cosine similarity


class EMA():
    # exponential moving average
    def __init__(self, beta):
        super().__init__()
        self.beta = beta

    def update_average(self, old, new):
        if old is None:
            return new
        return old * self.beta + (1 - self.beta) * new# fa interpolazione alla alpha way

def update_moving_average(ema_updater, ma_model, current_model):
    for current_params, ma_params in zip(current_model.parameters(), ma_model.parameters()):
        old_weight, up_weight = ma_params.data, current_params.data
        ma_params.data = ema_updater.update_average(old_weight, up_weight)

# MLP class for projector and predictor

def MLP(dim, projection_size, hidden_size=4096, sync_batchnorm=None):
    return nn.Sequential(
        nn.Linear(dim, hidden_size),
        nn.BatchNorm1d(hidden_size),
        nn.ReLU(inplace=True),
        nn.Linear(hidden_size, projection_size)
    )


class BYOL(nn.Module):# siamese net asimmetrica
    def __init__(self, backbone, moving_average_decay = 0.99):
        super().__init__()
        self.target_ema_updater = EMA(moving_average_decay) # update with exponential moving average
        self.target_net=None


        self.online_net = backbone # update with SGD
        self.online_net.fc = nn.Identity()
        self.online_projector = MLP(512, 512, 4096)

    def _get_target_encoder(self):
        if self.target_net is None:
            target_net = copy.deepcopy(self.online_net)
            for p in target_net.parameters():
                p.requires_grad = False
            self.target_net = target_net
        else:
            target_net = self.target_net
        return target_net

    def update_moving_average(self):
        update_moving_average(self.target_ema_updater, self.target_net, self.online_net)

    def forward(self, x1, x2):

        images = torch.cat((x1, x2), dim = 0)

        online_projections = self.online_projector(self.online_net(images))
        online_pred_one, online_pred_two = online_projections.chunk(2, dim = 0)

        with torch.no_grad():# 
            target_net = self._get_target_encoder()

            target_projections = target_net(images)
            target_projections = target_projections.detach()
            target_proj_one, target_proj_two = target_projections.chunk(2, dim = 0)

        loss_one = loss_fn(online_pred_one, target_proj_two.detach())# efficienza - èosso calcolare, con le 2 batch aumentate, posso calcolaarlo come 2 batch di dati, la prima volta apsso una vista 
        # e posso tranquillamente fare il contrario, passo la seconda vista a onlie e la prima la passo al target 

        # posso calcolarlio come se avessi 2 vatch

        # NON POSSO FARLO CON CONTRASTIVE LOSS: perché ogni volta guardo tutti i negative quindi non posso swappare perché sarebbe la stessa cosa
        #qui invece ogni istanza è indipendente
        loss_two = loss_fn(online_pred_two, target_proj_one.detach())

        loss = loss_one + loss_two
        return loss.mean()

## Exercise 0
PROB:
come fare stiopping gradient: una della 2 reti non ha il gradiente su di esse, c'è qualcos'altro da aggiungere. vedi lato pretrained network quando hai freezato i parameteri

come mai calcoli 2 loss?

nel report non ci metti ne tabelle ne grafi ma devo fare building block di ogni metodo e implementazione

puoi usare grafici fatti a mano che aiutano a spiegare quello che vogliamo dire e.g. power point o canva x accompagnare spiegazione

capacità do capire e esporre un metodo


# NB: da solo il modello tende a collasse, lo studente evita collapse perché segue rete che non è aggiornato con sgd



Study the above code.
- Where is the EMA updates? # dove interviene nel training, cosa e come freezzi il gradiente?
- Why it computes both loss_one and loss_two values? -> La loss è simmetrica, è somma di due modi diversi di dare le viste all'encoder. In Sto confrontanfdo le l'uscita della rete online su una trasformazione con l'uscita della rete target su un'altra trasformazione della stessa immagine (simil-positive)

# vogliamo che le reti imparino l'una dall'altra quindi la loss deve essere effettuata tra view diverse delle 2 reti - calcolo a croce. 

## Exercise 1

Write the training loop for moco-style training as used in BYOL.
Use the Dataset which creates the two augmented views for each image and the Siamese Network from the past lab session [1](https://colab.research.google.com/drive/1NJwAFbRiD4MdwWf__6P2Lm0xYk_DNdVu?usp=sharing) and [2](https://colab.research.google.com/drive/1AMkh0q8L5nJScx7v6cMWoK336zqOqDY6?usp=sharing).

# Commenti
prendo implementazione siamese network in cui ongline upodated with sgd and target EMA
# per la predizione prendi quello di online
il predittore lo devo scartare per evlautation

In [50]:
backbone = models.resnet18()
model = BYOL(backbone).to('cuda')
optimizer = optim.Adam(model.parameters(), lr=1e-3)


online_projector_param = {'params': model.online_projector.parameters()}
online_net_param = {'params': model.online_net.parameters()}


optimizer = torch.optim.SGD([online_net_param, online_projector_param], lr=0.4, momentum =0.9, weight_decay = 1e-04)
#2 torch.nograd che agisce su backward
#1 requiresgradfalse utile per optimizer step 
for idx, (x1,x2) in enumerate(dataloader):
    optimizer.zero_grad()
    x1 = x1.cuda()
    x2 = x2.cuda()
    loss = model(x1, x2)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    model.update_moving_average() # is it enough to update the target net?

    print('Indice: ', idx, ' Loss: ', loss.item())

    if idx == 3:
        break

Indice:  0  Loss:  4.095646858215332
Indice:  1  Loss:  1.4419841766357422
Indice:  2  Loss:  1.4254175424575806
Indice:  3  Loss:  1.4239393472671509


In [51]:
byol = BYOL(backbone).to('cuda')

